In [1]:
import pandas as pd
pd.set_option("display.max_row", None)

In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
deal_data = {
    "name":"Sample with Interest Swap"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{'assets':[["Mortgage"
                        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":2200
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]
            }
    ,"accounts":{"acc01":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}}
    ,"collect":[["CollectedInterest","acc01"]
              ,["CollectedPrincipal","acc01"]
              ,["CollectedPrepayment","acc01"]
              ,["CollectedRecoveries","acc01"]]
    ,"waterfall":{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]         
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [39]:
from absbox import API,mkDeal
localAPI = API("http://localhost:8081",check=False)

trigger = {
    "AfterCollect":
      {"poolDef":
        {"condition":[("cumPoolDefaultedBalance",),">",20]
        ,"effects":("newStatus","Defaulted")
        ,"status":False
        ,"curable":False}
      }
}

deal = mkDeal(deal_data|{"triggers":trigger})

r = localAPI.run(deal
                 ,poolAssump = ("Pool"
                                ,("Mortgage"
                                 ,{"CDR":0.02} ,None, None, None)
                                 ,None
                                 ,None)
                 ,runAssump = [("inspect"
                                ,("MonthEnd",("cumPoolDefaultedBalance",)))
                              ,("call",{"afterDate":"2022-09-01"})
                              ,("fireTrigger",[("2021-10-01","AfterCollect","poolDef")])
                              ]
                 ,read=True)

#r['pool']['flow']

✅Connected, local lib:0.20.2, server:0.22.5

In [40]:
r['result']['status']

,Date,From,To
0,2021-07-15,PreClosing,Amortizing
1,2021-10-01,Amortizing,Defaulted
2,2022-09-20,Defaulted,Called


In [38]:
r['result']['status']

,Date,From,To
0,2021-07-15,PreClosing,Amortizing
1,2021-12-31,Amortizing,Defaulted
2,2022-09-20,Defaulted,Called


In [36]:
r['result']['logs']

,Type,Comment
0,Warning,Bond A1 is not paid off
1,Warning,Bond B is not paid off
2,Warning,Account acc01 has cash to be distributed
3,Warning,"Trigger Overrided to True (2021-10-01,EndColle..."
4,Warning,No waterfall distribution found on date2021-10...
5,Warning,No waterfall distribution found on date2021-11...
6,Warning,No waterfall distribution found on date2021-12...
7,Warning,No waterfall distribution found on date2022-01...
8,Warning,No waterfall distribution found on date2022-02...
9,Warning,No waterfall distribution found on date2022-03...


In [126]:
a= [1,2,3]
b= ["A","B"]

[ (x,y) for x in a for y in b]

[(1, 'A'), (1, 'B'), (2, 'A'), (2, 'B'), (3, 'A'), (3, 'B')]

In [ ]:
"assets":[]
      ,'issuanceStat':{'IssuanceBalance':8000000001.8}
      ,'cashflow':
        [["2022-10-28",7703073351,296926650.8,39621199.84]
        ,["2022-11-28",7404682416,298390935.6,38239799.05]
        ,["2022-12-28",7068698297,335984118.4,36851240.88]
        ,["2023-01-28",6754588776,314109521.1,35270038.62]
        ,["2023-02-28",6484198542,270390234,33782877.37]
        ,["2023-03-28",6213847967,270350575.7,32518759.42]
        ,["2023-04-28",5948509320,265338646.5,31253463.48]
        ,["2023-05-28",5688705557,259803763.2,30007334.46]
        ,["2023-06-28",5428759464,259946092.9,28787675.09]
        ,["2023-07-28",5175901751,252857713.1,27564949.08]
        ,["2023-08-28",4928990804,246910947.3,26372771.15]
        ,["2023-09-28",4685294315,243696488.3,25204562.17]
        ,["2023-10-28",4448845810,236448505.8,24048073.72]
        ,["2023-11-28",4215182466,233663343.6,22921167.01]
        ,["2023-12-28",3985947347,229235118.7,21803634.86]
        ,["2024-01-28",3759360916,226586431.3,20702493.3]
        ,["2024-02-28",3538894353,220466563.4,19610562.99]
        ,["2024-03-28",3323113360,215780992.7,18542390.48]
        ,["2024-04-28",3113335758,209777601.8,17492785.73]
        ,["2024-05-28",2907923173,205412585.4,16466179.14]
        ,["2024-06-28",2703346681,204576491.7,15458135.54]
        ,["2024-07-28",2511363265,191983415.7,14454511.7]
        ,["2024-08-28",2328011947,183351318.5,13500336.9]
        ,["2024-09-28",2154558970,173452976.8,12579982.71]
        ,["2024-10-28",1987657739,166901231.6,11699767.21]
        ,["2024-11-28",1826757367,160900371.7,10848388.31]
        ,["2024-12-28",1672568344,154189022.8,10025127.3]
        ,["2025-01-28",1523301145,149267198.8,9233666.38]
        ,["2025-02-28",1391357178,131943966.8,8465915.05]
        ,["2025-03-28",1265990068,125367110.5,7781807.72]
        ,["2025-04-28",1148751455,117238613.1,7127147.05]
        ,["2025-05-28",1036382880,112368574.8,6509482.45]
        ,["2025-06-28",936394625.7,99988254.23,5914372.72]
        ,["2025-07-28",851012123.3,85382502.47,5381543.83]
        ,["2025-08-28",778711315.7,72300807.56,4921856.54]
        ,["2025-09-28",721110322.3,57600993.4,4521948.54]
        ,["2025-10-28",665295441.8,55814880.46,4193570.1]
        ,["2025-11-28",610860431.7,54435010.17,3874600.17]
        ,["2025-12-28",558006506.8,52853924.88,3562862.69]
        ,["2026-01-28",506302198.1,51704308.72,3259695.55]
        ,["2026-02-28",457913355.7,48388842.39,2962622.7]
        ,["2026-03-28",410739506.7,47173848.94,2683796.57]
        ,["2026-04-28",365271282,45468224.78,2411532.19]
        ,["2026-05-28",321236485.9,44034796.02,2148403.47]
        ,["2026-06-28",279263716.2,41972769.71,1893121.68]
        ,["2026-07-28",240177960.2,39085756.02,1648962.9]
        ,["2026-08-28",203790300.9,36387659.26,1420731.37]
        ,["2026-09-28",170218846,33571454.92,1206970.51]
        ,["2026-10-28",139622059.2,30596786.79,1008386.32]
        ,["2026-11-28",111766649.5,27855409.75,826711.04]
        ,["2026-12-28",86949691.98,24816957.49,661149.47]
        ,["2027-01-28",64671479.97,22278212,513593.06]
        ,["2027-02-28",48386675.38,16284804.59,381046.4]
        ,["2027-03-28",34169784.81,14216890.57,284519.48]
        ,["2027-04-28",22935717.42,11234067.39,200345.49]
        ,["2027-05-28",13354353.39,9581364.03,133877.2]
        ,["2027-06-28",6283905.88,7070447.52,77370.71]
        ,["2027-07-28",1919297.32,4364608.56,35979.93]
        ,["2027-08-28",0,1919297.32,10964.13]
      ]
     ,"extendBy":"MonthEnd"